# Advent of code 2022

In [1]:
import pandas as pd

## 01

In [31]:
def read_txt_file(path:str):
    with open(path) as f:
        string = f.read()
    return string

def solve_01(path:str, n=1):
    input_txt = read_txt_file(path)
    
    elf_food_sums = []
    for elf_food_str in input_txt.split("\n\n"):
        elf_food_int = [int(x) for x in elf_food_str.split("\n") if x != ""]
        elf_food_sums.append(sum(elf_food_int))
    return sum(sorted(elf_food_sums)[-n:])

### Example

In [15]:
example_path = "inputs/01_example.txt"

In [32]:
solve_01(example_path)

24000

In [33]:
solve_01(example_path, n=3)

45000

### Puzzle

In [34]:
input_file = "inputs/01_input.txt"

In [35]:
solve_01(input_file)

66616

In [36]:
solve_01(input_file, n=3)

199172

## 02

In [14]:
def solve_02(path:str):
    pass
#     input_txt = read_txt_file(path)
    
#     elf_food_sums = []
#     for elf_food_str in input_txt.split("\n\n"):
#         elf_food_int = [int(x) for x in elf_food_str.split("\n") if x != ""]
#         elf_food_sums.append(sum(elf_food_int))
#     return max(elf_food_sums)

### Example

In [15]:
example_path = "inputs/02_example.txt"

In [16]:
solve_02(example_path)

24000

### Puzzle

In [17]:
input_file = "inputs/02_input.txt"

In [20]:
solve_02(input_file)

66616